# Example Usage of the places_client

### 1. Import the package

In [1]:
from places_client.places_client import PlacesClient

### 2. Create a client for the API

In [2]:
# store your api token as "CDC_API_TOKEN" in the .env file
import os
from dotenv import load_dotenv
load_dotenv()

token = os.getenv('CDC_API_TOKEN')
client = PlacesClient(token)

### 3. Retrieve a certain release of filtered county-level PLACES data
`get_county_data()` retrieves county-level PLACES data from one of the six releases (2020, 2021, 2022, 2023, 2024, and 2025).  

It automatically filters the dataset to keep only “Health Outcomes” and “Health Risk Behaviors” measures.

In [ ]:
# get the 2024 release of county-level PLACES data
df_2024 = client.get_county_data(release="2024")
df_2024.tail(3)

,year,stateabbr,statedesc,locationname,datasource,category,measure,data_value_unit,data_value_type,data_value,low_confidence_limit,high_confidence_limit,totalpopulation,totalpop18plus,locationid,categoryid,measureid,datavaluetypeid,short_question_text,geolocation
100369,2021,WI,Wisconsin,Polk,BRFSS,Health Outcomes,High blood pressure among adults,%,Crude prevalence,34.3,29.9,38.8,45709,36755,55095,HLTHOUT,BPHIGH,CrdPrv,High Blood Pressure,"{'type': 'Point', 'coordinates': [-92.44127559..."
100370,2022,WI,Wisconsin,Trempealeau,BRFSS,Health Outcomes,Depression among adults,%,Age-adjusted prevalence,24.5,20.9,28.2,30899,23116,55121,HLTHOUT,DEPRESSION,AgeAdjPrv,Depression,"{'type': 'Point', 'coordinates': [-91.35842148..."
100371,2022,WY,Wyoming,Fremont,BRFSS,Health Risk Behaviors,Binge drinking among adults,%,Crude prevalence,16.4,13.5,19.5,39472,29818,56013,RISKBEH,BINGE,CrdPrv,Binge Drinking,"{'type': 'Point', 'coordinates': [-108.6304546..."


In [11]:
df_2024['measureid'].unique()

array(['CASTHMA', 'ARTHRITIS', 'STROKE', 'OBESITY', 'BINGE', 'BPHIGH',
       'DIABETES', 'DEPRESSION', 'TEETHLOST', 'COPD', 'CHD', 'LPA',
       'HIGHCHOL', 'CSMOKING', 'SLEEP', 'CANCER'], dtype=object)

### 4. Get an overview of supported measures

In [22]:
measures = client.get_measure_list()
measures[::4]

,id,short_name,full_name,category
0,ARTHRITIS,Arthritis,Arthritis among adults,Health Outcomes
4,CHD,Coronary Heart Disease,Coronary heart disease among adults,Health Outcomes
8,HIGHCHOL,High Cholesterol,High cholesterol among adults who have ever be...,Health Outcomes
12,TEETHLOST,All Teeth Lost,All teeth lost among adults aged >=65 years,Health Outcomes
16,SLEEP,Short Sleep Duration,Short sleep duration among adults,Health Risk Behaviors


### 5. Filter data by measures or categories
To filter by measures, provide either the short names of ids of measures. To filter by a category, use either "Health Outcomes" or "Health Risk Behaviors".

In [30]:
# filter by measure short names
sub_by_measure = client.filter_by_measures(df_2024, measures=["Coronary Heart Disease", "Current Asthma"])
# OR filter by measure ids
sub_by_measure = client.filter_by_measures(df_2024, measure_ids=["CHD", "CASTHMA"])
print(sub_by_measure['measureid'].unique())

['CASTHMA' 'CHD']


In [32]:
# filter by category
sub_by_category = client.filter_by_measures(df_2024, categories=['Health Outcomes'])
print(sub_by_category['category'].unique())
print(sub_by_category['measureid'].unique())

['Health Outcomes']
['CASTHMA' 'ARTHRITIS' 'STROKE' 'OBESITY' 'BPHIGH' 'DIABETES' 'DEPRESSION'
 'TEETHLOST' 'COPD' 'CHD' 'HIGHCHOL' 'CANCER']
